In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
 data = spark.read.csv("/FileStore/tables/movielens_ratings.csv", header = True, inferSchema= True)

In [0]:
display(data.show(1))


+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
+-------+------+------+
only showing top 1 row

In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [0]:
# hard to evaluate exclusively
train_data, test_data = data.randomSplit([0.8, 0.2])

In [0]:
als = ALS(maxIter = 5, regParam = 0.01, userCol = 'userId', itemCol = 'movieId', ratingCol = 'rating')

In [0]:
model = als.fit(train_data)

In [0]:
predictions = model.transform(test_data)

In [0]:
predictions.show() # predict rating *, negative means really dislike

+-------+------+------+------------+
movieId|rating|userId| prediction|
+-------+------+------+------------+
 31| 1.0| 4| 2.3614979|
 31| 3.0| 7| 1.5057538|
 31| 4.0| 12| 1.2398905|
 31| 1.0| 13| 1.5393516|
 31| 1.0| 18| -0.4238522|
 31| 1.0| 19| 0.18196693|
 31| 1.0| 27| 0.11321829|
 85| 1.0| 2| 0.80732936|
 85| 5.0| 8| 3.227004|
 85| 1.0| 29| -0.24244374|
 65| 1.0| 4| 1.5851214|
 65| 1.0| 19| 2.6591926|
 65| 1.0| 28| 1.1926994|
 53| 3.0| 13| 2.4436831|
 78| 1.0| 12| 1.1544514|
 78| 1.0| 20| 1.0799484|
 34| 4.0| 2|-0.083826765|
 34| 1.0| 4| 0.9230191|
 34| 1.0| 14| 1.1025282|
 34| 1.0| 15| 2.0656703|
+-------+------+------+------------+
only showing top 20 rows

In [0]:
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'rating')

In [0]:
rmse = evaluator.evaluate(predictions)

In [0]:
rmse   # this is pretty bad , rating 1-5, but this is a small dataset

Out[12]: 1.7427897772275893

In [0]:
single_user = test_data.filter(test_data['userId']== 11).select('movieId', 'userId')

In [0]:
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 23| 11|
 32| 11|
 36| 11|
 37| 11|
 69| 11|
 89| 11|
 99| 11|
+-------+------+

In [0]:
recommensations = model.transform(single_user) #predict and add them back

In [0]:
recommensations.orderBy('prediction', ascending = False).show() # based on previous review habit and similar users experience

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 36| 11| 1.8238715|
 23| 11| 1.7437003|
 99| 11| 0.96677274|
 69| 11| 0.956612|
 37| 11|-0.22349913|
 89| 11| -1.1995312|
 32| 11| -1.4993349|
+-------+------+-----------+

In [0]:
# user news? -> survey/ similar profile, 